In [1]:
import pynq
from dishsoap import StreamController, TestDmaStreamOverlay
streamer = TestDmaStreamOverlay()

In [2]:
results = pynq.allocate(shape=(125,), dtype='u8')
streamer.fill_buffer(5, results)
print(results)

[  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129]


## Benchmark

In [3]:
from time import time

def do_benchmark(size = 16, iters = 1) -> dict:
    t0 = time()
    bench_array = pynq.allocate(shape=(size,),dtype='u8')
    t1 = time()
    for _ in range(iters):
        streamer.fill_buffer(0, bench_array)
    t2 = time()
    return {'alloc': t1-t0, 'fill': t2-t1, 'total':t2-t0}


results = []

sizelist = [2**s for s in range(16)]
sizelist += [(2<<15) - 1] # also do max counter size
iterlist = [10**i for i in range(5)]

for size in sizelist:
    for iters in iterlist:
        times = do_benchmark(size, iters)
        times['size'] = size
        times['iters'] = iters
        results.append(times)
        print(f'Size: {size:>12,d}  Iters: {iters:6,}', end='  ')
        for category in ('alloc', 'fill', 'total'):
            print(f'{category}: {times[category]:10.6f}s', end='  ')
        print()


import pickle
with open('results.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)
    print(f"Saved results to 'results.pickle'")

Size:            1  Iters:      1  alloc:   0.031467s  fill:   0.000894s  total:   0.032361s  
Size:            1  Iters:     10  alloc:   0.035828s  fill:   0.005556s  total:   0.041384s  
Size:            1  Iters:    100  alloc:   0.036016s  fill:   0.049011s  total:   0.085027s  
Size:            1  Iters:  1,000  alloc:   0.034850s  fill:   0.488341s  total:   0.523191s  
Size:            1  Iters: 10,000  alloc:   0.035284s  fill:   4.725627s  total:   4.760911s  
Size:            2  Iters:      1  alloc:   0.035925s  fill:   0.000788s  total:   0.036713s  
Size:            2  Iters:     10  alloc:   0.036231s  fill:   0.005245s  total:   0.041476s  
Size:            2  Iters:    100  alloc:   0.035352s  fill:   0.049049s  total:   0.084401s  
Size:            2  Iters:  1,000  alloc:   0.039369s  fill:   0.474272s  total:   0.513641s  
Size:            2  Iters: 10,000  alloc:   0.035377s  fill:   4.701771s  total:   4.737148s  
Size:            4  Iters:      1  alloc:   0.0356

In [4]:
big_test = pynq.allocate(shape=(5000,), dtype='u8')
streamer.fill_buffer(60000,big_test)
print(big_test)

[60000 60001 60002 ... 64997 64998 64999]


## Results:

Speeds not including time to allocate the buffer on PS-side

In [5]:
print(f"{'iters':>15s} {'size (bytes)':>15s} {'speed (Mbps)':>15} {'(MB/s)':>15s}")
for res in results:
    bits = res['iters'] * res['size'] * 8
    mbps = bits / ((10**6) * res['fill'])
    MBs  = bits / ((2**20) * 8 * res['fill'])
    print(f"{res['iters']:>15,d} {res['size']*8:15,d} {mbps:15.3f} {MBs:15.3f} ")

          iters    size (bytes)    speed (Mbps)          (MB/s)
              1               8           0.009           0.001 
             10               8           0.014           0.002 
            100               8           0.016           0.002 
          1,000               8           0.016           0.002 
         10,000               8           0.017           0.002 
              1              16           0.020           0.002 
             10              16           0.031           0.004 
            100              16           0.033           0.004 
          1,000              16           0.034           0.004 
         10,000              16           0.034           0.004 
              1              32           0.044           0.005 
             10              32           0.059           0.007 
            100              32           0.066           0.008 
          1,000              32           0.066           0.008 
         10,000           